In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os
import time
import re
import math
from scipy import ndimage, misc

In [2]:
numbers=re.compile(r'(\d+)')
def numericalsort(value):
  
    parts=numbers.split(value)
    parts[1::2]=map(int, parts[1::2])
    return parts

In [3]:
os.chdir("C:/Users/Ying/Downloads/LBP_TOP")
orig_dir = os.getcwd()
orig_dir

'C:\\Users\\Ying\\Downloads\\LBP_TOP'

In [4]:
def hof(flow, orientations=9, pixels_per_cell=(8, 8),
        cells_per_block=(2, 2), normalise=False, motion_threshold=1.):

    """Extract Histogram of Optical Flow (HOF) for a given image.
    Key difference between this and HOG is that flow is MxNx2 instead of MxN
    Compute a Histogram of Optical Flow (HOF) by
        1. (optional) global image normalisation
        2. computing the dense optical flow
        3. computing flow histograms
        4. normalising across blocks
        5. flattening into a feature vector
    Parameters
    ----------
    Flow : (M, N) ndarray
        Input image (x and y flow images).
    orientations : int
        Number of orientation bins.
    pixels_per_cell : 2 tuple (int, int)
        Size (in pixels) of a cell.
    cells_per_block  : 2 tuple (int,int)
        Number of cells in each block.
    normalise : bool, optional
        Apply power law compression to normalise the image before
        processing.
    static_threshold : threshold for no motion
    Returns
    -------
    newarr : ndarray
        hof for the image as a 1D (flattened) array.
    hof_image : ndarray (if visualise=True)
        A visualisation of the hof image.
    References
    ----------
    * http://en.wikipedia.org/wiki/Histogram_of_oriented_gradients
    * Dalal, N and Triggs, B, Histograms of Oriented Gradients for
      Human Detection, IEEE Computer Society Conference on Computer
      Vision and Pattern Recognition 2005 San Diego, CA, USA
    """
    flow = np.atleast_2d(flow)

    """ 
    -1-
    The first stage applies an optional global image normalisation
    equalisation that is designed to reduce the influence of illumination
    effects. In practice we use gamma (power law) compression, either
    computing the square root or the log of each colour channel.
    Image texture strength is typically proportional to the local surface
    illumination so this compression helps to reduce the effects of local
    shadowing and illumination variations.
    """

    if flow.ndim < 3:
        raise ValueError("Requires dense flow in both directions")

    if normalise:
        flow = sqrt(flow)

    """ 
    -2-
    The second stage computes first order image gradients. These capture
    contour, silhouette and some texture information, while providing
    further resistance to illumination variations. The locally dominant
    colour channel is used, which provides colour invariance to a large
    extent. Variant methods may also include second order image derivatives,
    which act as primitive bar detectors - a useful feature for capturing,
    e.g. bar like structures in bicycles and limbs in humans.
    """

    if flow.dtype.kind == 'u':
        # convert uint image to float
        # to avoid problems with subtracting unsigned numbers in np.diff()
        flow = flow.astype('float')

    gx = np.zeros(flow.shape[:2])
    gy = np.zeros(flow.shape[:2])
    # gx[:, :-1] = np.diff(flow[:,:,1], n=1, axis=1)
    # gy[:-1, :] = np.diff(flow[:,:,0], n=1, axis=0)

    gx = flow[:,:,1]
    gy = flow[:,:,0]


    """ 
    -3-
    The third stage aims to produce an encoding that is sensitive to
    local image content while remaining resistant to small changes in
    pose or appearance. The adopted method pools gradient orientation
    information locally in the same way as the SIFT [Lowe 2004]
    feature. The image window is divided into small spatial regions,
    called "cells". For each cell we accumulate a local 1-D histogram
    of gradient or edge orientations over all the pixels in the
    cell. This combined cell-level 1-D histogram forms the basic
    "orientation histogram" representation. Each orientation histogram
    divides the gradient angle range into a fixed number of
    predetermined bins. The gradient magnitudes of the pixels in the
    cell are used to vote into the orientation histogram.
    """

    magnitude = np.sqrt(gx**2 + gy**2)
    orientation = np.arctan2(gy, gx) * (180 / math.pi) % 180

    sy, sx = flow.shape[:2]
    cx, cy = pixels_per_cell
    bx, by = cells_per_block

    n_cellsx = int(np.floor(sx // cx))  # number of cells in x
    n_cellsy = int(np.floor(sy // cy))  # number of cells in y

    # compute orientations integral images
    orientation_histogram = np.zeros((n_cellsy, n_cellsx, orientations))
    subsample = np.index_exp[int(cy / 2):cy * n_cellsy:cy, int(cx / 2):cx * n_cellsx:cx]
    for i in range(orientations-1):
        #create new integral image for this orientation
        # isolate orientations in this range

        temp_ori = np.where(orientation < 180 / orientations * (i + 1),
                            orientation, -1)
        temp_ori = np.where(orientation >= 180 / orientations * i,
                            temp_ori, -1)
        # select magnitudes for those orientations
        cond2 = (temp_ori > -1) * (magnitude > motion_threshold)
        temp_mag = np.where(cond2, magnitude, 0)

        #temp_filt = uniform_filter(temp_mag, size=(cy, cx))
        temp_filt = ndimage.uniform_filter(temp_mag, size=(cy, cx))
        orientation_histogram[:, :, i] = temp_filt[subsample]

    ''' Calculate the no-motion bin '''
    temp_mag = np.where(magnitude <= motion_threshold, magnitude, 0)

    temp_filt = ndimage.uniform_filter(temp_mag, size=(cy, cx))
    orientation_histogram[:, :, -1] = temp_filt[subsample]

    """
    The fourth stage computes normalisation, which takes local groups of
    cells and contrast normalises their overall responses before passing
    to next stage. Normalisation introduces better invariance to illumination,
    shadowing, and edge contrast. It is performed by accumulating a measure
    of local histogram "energy" over local groups of cells that we call
    "blocks". The result is used to normalise each cell in the block.
    Typically each individual cell is shared between several blocks, but
    its normalisations are block dependent and thus different. The cell
    thus appears several times in the final output vector with different
    normalisations. This may seem redundant but it improves the performance.
    We refer to the normalised block descriptors as Histogram of Oriented
    Gradient (hog) descriptors.
    """

    n_blocksx = (n_cellsx - bx) + 1
    n_blocksy = (n_cellsy - by) + 1
    normalised_blocks = np.zeros((n_blocksy, n_blocksx,
                                  by, bx, orientations))

    for x in range(n_blocksx):
        for y in range(n_blocksy):
            block = orientation_histogram[y:y+by, x:x+bx, :]
            eps = 1e-5
            normalised_blocks[y, x, :] = block / np.sqrt(block.sum()**2 + eps)

    return normalised_blocks.ravel()

In [5]:
#extract Histogram of Optical Flow features for a given consequetive frames. Replace frame by extracted HOG
#def extract_hof_feature(seq_location_list,seq_label):
def extract_hof_feature(seq_location):
    feature_hof = []
    label_list = []
    img_width = 128
    img_height = 64
        
    filepath=seq_location

    seq_folder = os.listdir(filepath)
    seq_folder_sorted=sorted(seq_folder, key=numericalsort)
    hof_hist=[]

    #iterate the frames inside a given sequence
    for b in range(0,len(seq_folder_sorted)-1):
        framepath = os.path.join(filepath, seq_folder_sorted[b])
        #print("image being processed", framepath)
        #read first frame
        frame = cv2.imread(framepath)
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
 

        previousGray = gray

        framepath_next = os.path.join(filepath, seq_folder_sorted[b+1])

        frame_next = cv2.imread(framepath_next)
 

        gray = cv2.cvtColor(frame_next, cv2.COLOR_BGR2GRAY)
 


        flow = cv2.calcOpticalFlowFarneback(previousGray, gray, 
                                            flow=None, pyr_scale=0.5, levels=5, winsize=11, 
                                            iterations=10, poly_n=5, poly_sigma=1.1, flags=0)

        hof_feature_one = hof(flow, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2, 2))


        hof_hist.append(hof_feature_one)        
    
    return(hof_hist)



In [6]:
###### EXTRACT HOF FEATURE

frames_source_path=os.path.join(orig_dir, 'small_Cropped')
subject_list = os.listdir(frames_source_path)
sub_subject_list=subject_list[:]

for subject in sub_subject_list:

    subject_path_list=os.path.join(frames_source_path, subject)
    file_list = os.listdir(subject_path_list)
    for folder in file_list:
        folder_file=os.path.join(subject_path_list, folder)
        frame_list=os.listdir(folder_file)
                     
        # extract vectors
        # Get the frame volume sequence
        count=0
        start_time=time.time()
        hist_hof=extract_hof_feature(folder_file)
        os.chdir("C:/Users/Ying/Downloads/LBP_TOP/HOF_cropped_raw")
        location=os.getcwd()
        new_subject_path=os.path.join(location, subject)
        if not os.path.exists(new_subject_path):
            os.mkdir(new_subject_path)
        os.chdir(new_subject_path)
        if not os.path.exists(folder):
            os.mkdir(folder)
            print ("created subdirectory for folder", folder)
        new_folder_path=os.path.join(new_subject_path, folder)
        os.chdir(new_folder_path)
        print (" at file saving location", new_folder_path)
        filename="hof_full.npy"
        save_file=os.path.join(new_folder_path, filename)
        location=save_file
        np.save(save_file,hist_hof)

            
            
os.chdir(orig_dir)   

 at file saving location C:\Users\Ying\Downloads\LBP_TOP\HOF_cropped_raw\sub01\EP02_01f
 at file saving location C:\Users\Ying\Downloads\LBP_TOP\HOF_cropped_raw\sub01\EP03_02
 at file saving location C:\Users\Ying\Downloads\LBP_TOP\HOF_cropped_raw\sub01\EP04_02
 at file saving location C:\Users\Ying\Downloads\LBP_TOP\HOF_cropped_raw\sub01\EP04_03
 at file saving location C:\Users\Ying\Downloads\LBP_TOP\HOF_cropped_raw\sub01\EP04_04
 at file saving location C:\Users\Ying\Downloads\LBP_TOP\HOF_cropped_raw\sub01\EP19_01
 at file saving location C:\Users\Ying\Downloads\LBP_TOP\HOF_cropped_raw\sub01\EP19_03f
 at file saving location C:\Users\Ying\Downloads\LBP_TOP\HOF_cropped_raw\sub01\EP19_05f
 at file saving location C:\Users\Ying\Downloads\LBP_TOP\HOF_cropped_raw\sub01\EP19_06f
 at file saving location C:\Users\Ying\Downloads\LBP_TOP\HOF_cropped_raw\sub02\EP01_11f
 at file saving location C:\Users\Ying\Downloads\LBP_TOP\HOF_cropped_raw\sub02\EP02_04f
 at file saving location C:\Users\Yin